# Slice Sampling

Slice sampling {cite}`neal2003slice` is a Markov Chain Monte Carlo (MCMC) method for sampling from unnormalised probability distributions, developed by Radford Neal. Similarly to all other MCMC methods, Slice Sampling starts out from an initial point $x_0$ and evolves this according to a rule $p(x_{n + 1} | x_n)$, such that the distribution of $x_n$ approaches $\pi$ in the limit $n \to \infty$.

The feature that makes Slice Sampling interesting is that it involves very few tunable parameters and virtually no critical design choices - unlike other sampling algorithms such as Metropolis Hastings, where the choice of proposal distribution can greatly affect the performance of the algorithm.

In [17]:
from IPython.display import HTML, set_matplotlib_formats

set_matplotlib_formats('pdf', 'svg')
css_style = open('../../../_static/custom_style.css', 'r').read()
HTML(f'<style>{css_style}</style>')

## The Slice Sampling algorithm

<div class='definition'>

**Definition (One dimensional slice sampling)** Given an unnormalised target distribution $f(x)$ and an initial point $x_0$, slice sampling generates a new point $x_1$ as follows:
    
1. Sample $y \sim \text{Uniform}[0, f(x_0)]$. This defines the slice set $$S_y = \{x : y = f(x)\}.$$
2. Generate a finite interval $I$ that contains $x_0$.
3. Sample $x_1$ uniformly from the set of intervals $S_y \cap I \cap A$, where
    
$$A = \{x_1 : p(x_1 | x_0, y) = p(x_0 | x_1, y)\}.$$
    
We write $p(x_1 | x_0)$ for the probability distribution of the new point $x_1$, conditioned on the previous point $x_0$.
    
</div>
<br>
    
For the second step of the above algorithm, Neal presents two expansion methods for generating $I$, the step-out method and the doubling method. For the third step, he shows that sampling from $S_y \cap I \cap A$ can be carried out efficiently. We'll look at the step-out and the doubling methods in more detail and provide proofs of the related results.

Note also that the above algorithm is a special case of the more general "crumb" framework.{cite}`neal2003slice` Crumb-based slice sampling is not constained to generating an $I$ and sampling from a subset of it, although methods that do so can be cast as crumb-based methods. Crumb-based methods are very general and therefore ammeanable to a great deal of tuning and design choices, which are beyond the scope of this page, so we won't look into them.

It can be shown that the sampling algorithm defined above leaves the target distribution invariant. To show this, it is enough to show that slice sampling satisfies detailed balance.

<div class='lemma'>

**Result (Slice sampling satisfies detailed balance)** The slice-sampling transition rule satisfies detailed balance
    
$$\begin{align}
p(x_1 | x_0) \pi(x_0) = p(x_0 | x_1) \pi(x_1).
\end{align}$$
    
</div>
<br>


<details class="proof">
<summary> Slice sampling satisfies detailed balance </summary>

We will show that slice sampling satisfies detailed balance of the target distribution. We have
$$\begin{align}
p(x_1 | x_0)\pi(x_0) &= \int_0^{\pi(x_0)} p(x_1 | x_0, y) p(y | x_0) \pi(x_0) dy \\
                     &= \int_0^{\pi(x_0)} p(x_1 | x_0, y) \frac{1}{\pi(x_0)} \pi(x_0) dy \\
                     &= \int_0^{\min(\pi(x_0), \pi(x_1))} p(x_1 | x_0, y) dy,
\end{align}$$
where in the last line we have used the fact that
$$\begin{align}
p(x_1 | x_0, y) = 0, \text{ whenever } y > \pi(x_1).
\end{align}$$
    
Similarly, going the other way we have
$$\begin{align}
p(x_0 | x_1)\pi(x_1) &= \int_0^{\pi(x_1)} p(x_0 | x_1, y) p(y | x_1) \pi(x_1) dy \\
                     &= \int_0^{\pi(x_1)} p(x_0 | x_1, y) \frac{1}{\pi(x_1)} \pi(x_1) dy \\
                     &= \int_0^{\min(\pi(x_0), \pi(x_1))} p(x_0 | x_1, y) dy.
\end{align}$$
Therefore, we only need to show that
$$\begin{align}
p(x_1 | x_0, y) &= p(x_0 | x_1, y) dy,
\end{align}$$
which is follows by the definition of $A$, that is
$$A = \{x_1 : p(x_1 | x_0, y) = p(x_0 | x_1, y)\}.$$
This proof in offloads a bit of the work to finding a way to ensure that $x$ lies not only in $S_y \cap I$ but also in $A$. We discuss this later.
    
</details>
<br>

## Interval-expansion methods

Neal proposes two methods for computing the finite interval $I$: the step-out method and the doubling method. We discuss each in turn.

### Step-out method

<div class='definition'>

**Definition (Step-out method)** Given $f, x_0, y$, positive integer $m$ and positive real $w > 0$, the step-out method produces an interval $I = [L, R]$ containing $x_0$ as follows
    
1. Sample $U, V \sim \text{Uniform}[0, 1]$.
2. Initialise endpoints $L = x_0 - wU$ and $R = x_0 + w$, and budgets $N_L = \text{floor}(mV)$ and $N_R = (m - 1) - N_L$.
3. While $N_L > 0$ and $y \leq f(L)$ update $L \leftarrow L - w$, $N_L \leftarrow N_L - 1$.
4. While $N_R > 0$ and $y \leq f(R)$ update $R \leftarrow R + w$, $N_R \leftarrow N_R - 1$.
    
</div>
<br>

This process expands $I$ to the left and to the right, terminating the expansion in each direction when it finds an endpoint outside the slice, or when the budget is used up. Selecting $U, V$ at random is of crucial importance in ensuring that the resulting $A$ is not too restrictive.

In [10]:
def step_out_expansion(x0, y, log_prob, w, m):
    
    U = np.random.rand()
    L = x0 - w * U
    R = L + w
    
    Lhist = [L]
    Rhist = [R]
    
    J = np.floor(m * np.random.rand())
    K = m - 1 - J
    
    fL = log_prob(L)
    fR = log_prob(R)
    
    while J > 0 and y <= fL:
        
        L = L - w
        J = J - 1
        fL = log_prob(L)
        
        Lhist.append(L)
        
    while K > 0 and y <= fR:
        
        R = R + w
        K = K - 1
        fR = log_prob(R)
        
        Rhist.append(R)
        
    return L, R, (Lhist, Rhist)

### Doubling method

<div class='definition'>

**Definition (Doubling method)** Given $f, x_0, y$, positive integer $p$ and positive real $w > 0$, the step-out method produces an interval $I = [L, R]$ containing $x_0$, as follows
    
1. Sample $U \sim \text{Uniform}[0, 1]$.
2. Initialise endpoints $L = x_0 - wU$ and $R = x_0 + w$ and budget $K = p$.
3. While $K > 0$ and $(y < f(L) \text{ or } y < f(R))$, sample $V \sim \text{Uniform}[0, 1]$ and update 
    
    $$\begin{align}
L \leftarrow L - (R - L) &&\text{ if } V < \frac{1}{2} \\
R \leftarrow R + (R - L) &&\text{ otherwise}.
\end{align}$$
    
</div>
<br>

In [11]:
def double_expansion(x0, y, log_prob, w, p):
    
    U = np.random.unform()
    L = x0 - w * U
    R = L + w
    K = p
    
    Lhist = [L]
    Rhist = [R]
    
    fL = log_prob(L)
    fR = log_prob(R)
    
    while K > 0 and (fL > y or fR > y):
        
        V = np.random.uniform()
        
        if V > 0.5:
            L = L - (R - L)
            fL = log_prob(L)
            
        else:
            R = R + (R - L)
            fR = log_prob(R)
        
        Lhist.append(L)
        Rhist.append(R)
        
        K = K - 1
            
    return L, R, (Lhist, Rhist)

## Interval sampling

### Sampling method

<div class='lemma'>

**Result (Acceptable sets for step-out/doubling methods)** Starting from the point $x_0$, the acceptable set
    
$$A = \{x_1 : p(x_1 | x_0, y) = p(x_0 | x_1, y)\}$$
    
is equal to $S \cap I$ for the step-out method, where $I$ is the interval expanded from $x_0$ using parameters $w, p$. For the doubling method, we can determine whether a candidate point $x_1$ is in the acceptable set defined by $x_0, y, w, p, I$ as follows
    
1. Initialise $[L, R] = I$.
2. While $R - L > w$ repeat

    2.1 Set $M = (L + R)~/~2$, and update $L, R$ according to
    
    $$\begin{align}
L \leftarrow M &&\text{ if } M \geq x_0 \\
R \leftarrow M &&\text{ otherwise}.
\end{align}$$
    
    2.2 If $y \geq f(L)$ and $y \geq f(R)$, $x$ is not acceptable.
3. If $x$ is not rejected in the above loop, it is acceptable.
    
</div>
<br>

In [18]:
def sample_acceptable_set(x0, y, L, R, log_prob, method, params, shrinkage):
    
    L0, R0 = L, R
    
    while True:
        
        x = L + (R - L) * np.random.rand()
        
        accept = is_acceptable(x, y, L0, R0, log_prob, method, params)
        
        if accept:
            return x
        
        elif shrinkage and x < x0:
            L = x
            
        elif shrinkage and x > x0:
            R = x

### Checking for acceptibility

In [19]:
def is_acceptable(x, y, L, R, log_prob, method, params):
    
    # Function value at candidate point
    f = log_prob(x)
    
    # For stepping out, check x is in the slice (y < f)
    if method == 'step_out':
        
        if y <= f:
            return True
        
        else:
            return False
    
    # For doubling, check the doubling sequence can be recreated the other way
    elif method == 'double':
        
        w, p = params
        
        fx = log_prob(x)
        
        while R - L > 1.5 * w:
            
            M = (R + L) / 2
            
            fL = log_prob(L)
            fM = log_prob(M)
            fR = log_prob(R)
            
            if x >= M and (y < fM or y < fR): L = M
                
            elif x < M and (y < fL or y < fM): R = M
            
            else: return False
            
        # If the point is not rejcected in the loop, it is acceptable
        return True

## The Slice Sampling algorithm

Putting it all together, we have a complete MCMC algorithm for sampling from $1$-dimensional distributions.

In [20]:
def sample(x0, log_prob, method, params, shrinkage):
    
    f0 = log_prob(x0)
    y = np.log(np.exp(f0) * np.random.rand())
    
    L = None
    R = None
    
    if method == 'step_out':
        w, m = params
        L, R, _ = step_out_expansion(x0, y, log_prob, w, m)
        
    elif method == 'double':
        w, p = params
        L, R, _ = double_expansion(x0, y, log_prob, w, p)
        
    x = sample_acceptable_set(x0=x0,
                              y=y,
                              L=L,
                              R=R,
                              log_prob=log_prob,
                              method=method,
                              params=params,
                              shrinkage=shrinkage)
    
    return x, y

## References

```{bibliography} ./references.bib
```